# Cleaning the Data

### First import the libraries and data as a dataframe

In [1]:
import pandas as pd
import numpy as np
import re # used for pattern matching and string manipulation in date column

In [ ]:
um_running_df = um_running_df = pd.read_csv("/Users/idsg/Desktop/Coding/daimil10/My_midterm/TWO_CENTURIES_OF_UM_RACES.csv")

### Begin cleaning the first file from Kaggle

### This code below was utilized to pull all events located within the United States


In [ ]:

usa_running_df = um_running_df[um_running_df['Event name'].str.contains("(USA)", case=True, na=False)]


### This filter removed all Men and Women age categories into a combined age category due to there being a column associating Male and Female

In [ ]:

usa_running_df['Athlete age category'] = usa_running_df['Athlete age category'].str.replace('M', '')
usa_running_df['Athlete age category'] = usa_running_df['Athlete age category'].str.replace('W', '')

### This gets rid of the 'h' in the athelete peformance time to format as a datetime later


In [ ]:

usa_running_df['Athlete performance'] = usa_running_df['Athlete performance'].str.replace(' h', '')

### Next, I cleaned all NaNs in the birth year so they were all integers, NaNs recieved a 0


In [ ]:

usa_running_df['Athlete year of birth'] = usa_running_df['Athlete year of birth'].fillna(0).astype(int)


# The next code allowed for the creation of a true age from events that had the year of and the birth year of the athlete to provide a true age along with the age category

In [ ]:


usa_running_df['true age'] = usa_running_df['Year of event'] - usa_running_df['Athlete year of birth']


### import the extra data from the csv UM_Race_Information.csv


In [8]:

event_info_df = pd.read_csv('/Users/idsg/Desktop/Coding/daimil10/My_midterm/Midterm_Running/Data/UM_Race_Information.csv')

### merge the data sets on Event name due to 1 for 1 matches


In [9]:

merged_data_df = pd.merge(usa_running_df, event_info_df, on='Event name', how='left')

### now drop merged data that is blank and only has the 100 events we selected


In [10]:

merged_data_df = merged_data_df.dropna(subset='Distance (miles)')

### This definition turn athlete performance time into total hours


In [11]:

def time_to_hours(time_str):
    if 'd' in time_str:
        d, time = time_str.split('d ')
        h, m, s = map(int, time.split(':'))
        return int(d) * 24 + h + m / 60 + s / 3600
    else:
        h, m, s = map(int, time_str.split(':'))
        return h + m / 60 + s / 3600

merged_data_df['Hours Ran'] = merged_data_df['Athlete performance'].apply(time_to_hours)

### from the distance of the race and total hours ran we get miles per hour or speed of a racer

In [12]:


merged_data_df['mph'] = merged_data_df['Distance (miles)'] / merged_data_df['Hours Ran']


### I dropped incomplete data that would skew my results


In [13]:

merged_data_df = merged_data_df.dropna(subset=['Athlete age category'])


### State abbreviations were added to utilize choropleth and map data


In [14]:

state_abbreviation = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

merged_data_df['state_code'] = merged_data_df['State'].map(state_abbreviation)


### regex was used in the Event dates column due to dates being formatted in multiple different ways
### from this definition the dates were able to format properly and a new datetime column was created with proper date time format

In [15]:


def clean_date(date):
    # Extract the pattern dd.mm.yyyy from the string
    match = re.search(r'\b\d{2}\.\d{2}\.\d{4}\b', date)
    if match:
        return match.group(0)
    return None

# Apply the function to the 'date' column
merged_data_df['cleaned_date'] = merged_data_df['Event dates'].apply(clean_date)

# Convert to datetime
merged_data_df['datetime'] = pd.to_datetime(merged_data_df['cleaned_date'], format='%d.%m.%Y', errors='coerce')

### Look at the data and remove any columns not needed.

In [ ]:
um_merged_df = merged_data_df.drop(columns=["Event distance/length", 'Event dates', 'Unnamed: 0', 'Athlete average speed', 'cleaned_date'])


### Write the finished and completed data frame to a csv

In [18]:
um_merged_df.to_csv('UM_data_cleaned.csv')